In [107]:
import pandas as pd
import numpy as np
from os.path import join
import requests
import sqlite3

In [108]:
from peewee import SqliteDatabase, fn, JOIN
from playhouse.reflection import generate_models, print_model, print_table_sql

In [109]:
db = SqliteDatabase('star.db')
models = generate_models(db)

globals().update({
    "Paper": models['papers'],
    "DomainPaperToField": models['domain_paper_to_field'],
    "DomainPaperToMethod": models['domain_paper_to_method']
})

In [110]:
# Get unique fields
unique_field_rows = (
    DomainPaperToField
        .select(DomainPaperToField.field)
        .distinct()
)
unique_fields = [ row.field for row in unique_field_rows ]
unique_fields

['Biology',
 'Economics',
 'Environmental Science',
 'Medicine',
 'Computer Science',
 'Physics',
 'Education',
 'Materials Science',
 'Psychology',
 'Chemistry',
 'Business',
 'Mathematics',
 'Sociology',
 'Political Science',
 'Philosophy',
 'Geology',
 'Engineering',
 'Geography',
 'Art',
 'History',
 'Agricultural And Food Sciences',
 'Linguistics',
 'Law']

In [111]:
field_count_rows = (
    DomainPaperToField
        .select(DomainPaperToField.field, fn.COUNT(DomainPaperToField.ss_id).alias('paper_count'))
        .group_by(DomainPaperToField.field)
)

field_count_df = pd.DataFrame(data=[ { "field": row.field, "paper_count": row.paper_count } for row in field_count_rows ])
field_count_df

,field,paper_count
0,Agricultural And Food Sciences,235
1,Art,259
2,Biology,8455
3,Business,747
4,Chemistry,1729
5,Computer Science,15856
6,Economics,1092
7,Education,140
8,Engineering,947
9,Environmental Science,2012


In [112]:
field_count_df.to_csv('papers_by_field.csv')

In [113]:
venue_count_rows = (
    Paper
        .select(Paper.venue, fn.COUNT(Paper.ss_id).alias('paper_count'))
        .group_by(Paper.venue)
)

venue_count_df = pd.DataFrame(data=[ { "venue": row.venue, "paper_count": row.paper_count } for row in venue_count_rows ])
venue_count_df = venue_count_df.replace('', np.nan).dropna(subset=['venue'])
venue_count_df = venue_count_df.sort_values(by='paper_count', ascending=False)
venue_count_df.loc[venue_count_df['paper_count'] >= 2].to_csv('papers_by_venue.csv')

In [114]:
method_field_rows = (DomainPaperToField
         .select(DomainPaperToField.field, DomainPaperToMethod.method_acronym, fn.COUNT(DomainPaperToField.ss_id).alias('paper_count'))
         .join(DomainPaperToMethod, on=(DomainPaperToField.ss_id == DomainPaperToMethod.ss_id))
         .group_by(DomainPaperToField.field, DomainPaperToMethod.method_acronym)
)

method_field_df = pd.DataFrame(data=[
    {
        "field": row.field,
        "method_acronym": row.domain_paper_to_method.method_acronym,
        "paper_count": row.paper_count
    } for row in method_field_rows
])
method_field_df.to_csv('papers_by_field_and_method.csv')

In [115]:
method_venue_rows = (DomainPaperToMethod
         .select(DomainPaperToMethod.method_acronym, Paper.venue, fn.COUNT(Paper.ss_id).alias('paper_count'))
         .join(Paper, on=(DomainPaperToMethod.ss_id == Paper.ss_id))
         .group_by(Paper.venue, DomainPaperToMethod.method_acronym)
)

method_venue_df = pd.DataFrame(data=[
    {
        "venue": row.papers.venue,
        "method_acronym": row.method_acronym,
        "paper_count": row.paper_count
    } for row in method_venue_rows
])
method_venue_df = method_venue_df.replace('', np.nan).dropna(subset=['venue'])
method_venue_df.to_csv('papers_by_venue_and_method.csv')